# Scraping Data Tables from Natural Stat Trick Using Beautiful Soup #

In this post, I will be demonstrating how to grab data from tables on the website [Natural Stat Trick](naturalstattrick.com). Natural Stat Trick is a website for getting standard and advanced NHL statistics. It is one of the more well known and frequently referenced sources of data in the NHL analytics world for writers and fans alike. 

First, using the Beautiful Soup library, I'll go through the steps to scrape the html and convert the data to a Pandas DataFrame. Then, I'll show a convenient one liner that works for tables on Natural Stat Trick and will work for tables on other webpages depending on their complexity. Finally, a tip on altering the URL to change the parameters of the data you can get.



## Beautiful Soup Step By Step ##

Import the below libraries.

In [142]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

The requests.get() method will download a webpage and return a response object

In [143]:
url = "http://www.naturalstattrick.com/playerteams.php?fromseason=20202021&thruseason=20202021&stype=2&sit=5v5&score=all&stdoi=oi&rate=n&team=ALL&pos=S&loc=B&toi=0&gpfilt=none&fd=&td=&tgp=410&lines=single&draftteam=ALL"
req = requests.get(url)

To confirm it worked with no errors, we can run the .status_code method. We are looking for this to return 200 which means we are all good. [HTML Codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes)

In [144]:
#200 is success
req.status_code

200

Here we are passing the text of the html from the request into a Beautiful Soup object so we can use the Beautiful Soup library to parse it.

In [145]:
soup = BeautifulSoup(req.content)

We use the .children method to work through the levels of child nodes in the html. We'll pass it to a list to access different levels. For this webpage there is only 1 child at the top level

In [146]:
top = list(soup.children)
len(top)

1

At the second level, there are two children: header and body. The contents of the table will be in the body which is found at index 1.

In [147]:
body = list(top[0].children)[1]


Use the .find_all method to filter by all the data that has a specified html tag and put it into a list. We want  all the 'th' tags to get the table headers. 

In [148]:
body.find_all('th')[0:5]

[<th></th>,
 <th class="lh">Player</th>,
 <th>Team</th>,
 <th>Position</th>,
 <th>GP</th>]

The .text method will extract the text within the tags. We'll pass the columns text into a list to use later.

In [149]:
columns = [item.text for item in body.find_all('th')]
columns[0:5]
    

['', 'Player', 'Team', 'Position', 'GP']

Now we'll find all the 'td' tags where the data is contained

In [150]:
body.find_all('td')[0:5]

[<td>1</td>,
 <td class="lh"><a href="playerreport.php?fromseason=20202021&amp;thruseason=20202021&amp;playerid=8465009&amp;sit=5v5&amp;stype=2">Zdeno Chara</a></td>,
 <td>WSH</td>,
 <td>D</td>,
 <td>23</td>]

And pass the text of the data into a list 

In [151]:
data = [e.text for e in body.find_all('td')]

In [152]:
data[0:5]

['1', 'Zdeno Chara', 'WSH', 'D', '23']

Here I will loop through the list of data to create a list of lists for each row of data in the table. Later Ill use this to create a pandas dataframe.

In [153]:
start = 0
table= []
#loop through entire data
while start+len(columns) <= len(data):
    player = []
    #use length of columns as iteration stop point to get list of info for 1 player 
    for i in range(start,start+len(columns)):
        player.append(data[i])
    #add player row to list
    table.append(player)
    #start at next player
    start += len(columns)

In [154]:
table[0:5]

[['1',
  'Zdeno Chara',
  'WSH',
  'D',
  '23',
  '357.5',
  '312',
  '335',
  '48.22',
  '225',
  '260',
  '46.39',
  '162',
  '180',
  '47.37',
  '19',
  '13',
  '59.38',
  '13.1',
  '12.03',
  '52.13',
  '143',
  '141',
  '50.35',
  '67',
  '54',
  '55.37',
  '11',
  '7',
  '61.11',
  '76',
  '87',
  '46.63',
  '4',
  '4',
  '50.00',
  '147',
  '169',
  '46.52',
  '4',
  '1',
  '80.00',
  '11.73',
  '92.78',
  '1.045',
  '38',
  '75',
  '46',
  '266',
  '45.24',
  '77',
  '110',
  '101',
  '43.26'],
 ['2',
  'Joe Thornton',
  'TOR',
  'C',
  '13',
  '167.2',
  '147',
  '126',
  '53.85',
  '105',
  '99',
  '51.47',
  '75',
  '72',
  '51.02',
  '9',
  '5',
  '64.29',
  '6.16',
  '5.3',
  '53.75',
  '92',
  '60',
  '60.53',
  '25',
  '25',
  '50.00',
  '5',
  '4',
  '55.56',
  '67',
  '35',
  '65.69',
  '4',
  '0',
  '100.00',
  '51',
  '56',
  '47.66',
  '0',
  '1',
  '0.00',
  '12.00',
  '93.06',
  '1.051',
  '41',
  '55',
  '21',
  '104',
  '66.13',
  '63',
  '74',
  '36',
  '63.64'

In [155]:
df = pd.DataFrame(table, columns = columns, dtype = 'float').set_index('')


And VOILA! I have now extraced the data from the webpage into a Pandas DataFrame. Now I can get to work cleaning and analyzing.

In [156]:
df.head()

,Player,Team,Position,GP,TOI,CF,CA,CF%,FF,FA,...,PDO,Off. Zone Starts,Neu. Zone Starts,Def. Zone Starts,On The Fly Starts,Off. Zone Start %,Off. Zone Faceoffs,Neu. Zone Faceoffs,Def. Zone Faceoffs,Off. Zone Faceoff %
,,,,,,,,,,,,,,,,,,,,,
1.0,Zdeno Chara,WSH,D,23.0,357.500000,312.0,335.0,48.22,225.0,260.0,...,1.045,38.0,75.0,46.0,266.0,45.24,77.0,110.0,101.0,43.26
2.0,Joe Thornton,TOR,C,13.0,167.200000,147.0,126.0,53.85,105.0,99.0,...,1.051,41.0,55.0,21.0,104.0,66.13,63.0,74.0,36.0,63.64
3.0,Patrick Marleau,S.J,C,21.0,229.416667,182.0,245.0,42.62,132.0,190.0,...,0.982,28.0,42.0,32.0,195.0,46.67,46.0,58.0,60.0,43.40
4.0,Jason Spezza,TOR,C,24.0,196.800000,159.0,175.0,47.60,128.0,132.0,...,1.028,33.0,29.0,26.0,195.0,55.93,53.0,46.0,48.0,52.48
5.0,Mikko Koivu,CBJ,C,7.0,77.150000,49.0,82.0,37.40,37.0,62.0,...,1.010,11.0,20.0,16.0,57.0,40.74,18.0,29.0,26.0,40.91


## The One Liner ##

Now as mentioned, we can do all that scraping, searching and transforming to a DataFrame in one concise line of code!
Pandas .read_html method reads HTML tables into a list of DataFrame objects and it's built on BeautifulSoup4 and lxml. It can also cast the columns into the appropriate data types. 

In [157]:
df2 = pd.read_html(url, header=0, index_col = 0, na_values=["-"])[0]

In [158]:
df2.head()

,Player,Team,Position,GP,TOI,CF,CA,CF%,FF,FA,...,PDO,Off. Zone Starts,Neu. Zone Starts,Def. Zone Starts,On The Fly Starts,Off. Zone Start %,Off. Zone Faceoffs,Neu. Zone Faceoffs,Def. Zone Faceoffs,Off. Zone Faceoff %
1,Zdeno Chara,WSH,D,23,357.500000,312,335,48.22,225,260,...,1.045,38,75,46,266,45.24,77,110,101,43.26
2,Joe Thornton,TOR,C,13,167.200000,147,126,53.85,105,99,...,1.051,41,55,21,104,66.13,63,74,36,63.64
3,Patrick Marleau,S.J,C,21,229.416667,182,245,42.62,132,190,...,0.982,28,42,32,195,46.67,46,58,60,43.40
4,Jason Spezza,TOR,C,24,196.800000,159,175,47.60,128,132,...,1.028,33,29,26,195,55.93,53,46,48,52.48
5,Mikko Koivu,CBJ,C,7,77.150000,49,82,37.40,37,62,...,1.010,11,20,16,57,40.74,18,29,26,40.91


In [159]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 798 entries, 1 to 798
Data columns (total 53 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Player               798 non-null    object 
 1   Team                 798 non-null    object 
 2   Position             798 non-null    object 
 3   GP                   798 non-null    int64  
 4   TOI                  798 non-null    float64
 5   CF                   798 non-null    int64  
 6   CA                   798 non-null    int64  
 7   CF%                  798 non-null    float64
 8   FF                   798 non-null    int64  
 9   FA                   798 non-null    int64  
 10  FF%                  798 non-null    float64
 11  SF                   798 non-null    int64  
 12  SA                   798 non-null    int64  
 13  SF%                  798 non-null    float64
 14  GF                   798 non-null    int64  
 15  GA                   798 non-null    int

## A Note on URL Parameters ##
By altering elements in the URL that you pass in the code, you can change what data comes back in the table. This allows for dynamically pulling different types of data. For example, you can switch to different seasons, dates, look at goalie stats, look at rates instead of counts, filter by different game situations etc... 


I've pulled out all the potential paremeters and noted different options below. Using the python method .format, we can change the parameters in the URL string. Compared to the URL string we were previously working with above, I'll change stdoi to 'g' to get goalie specific stats to demonstrate

In [160]:
# URL params
fromseason = 20202021
thruseason = 20202021
stype = 2 #preseason = 1 , regular season = 2, RTP exhibition = 3, playoffs = 4
sit = '5v5' #All strengths = all, even strength = ev, 5v5 score and venue adjusted = sva, power play = pp, 5 on 4 pp = 5v4, penalty kill = pk, 4 on 5 pk = 4v5, with empty net = enf, against empty net = ena
score = 'all' #All scores = all, tied = tied, leading = u, trailing = d, within 1 = w1, up 1 = u1, down 1 = d1
stdoi = 'g' #on ice = oi, individual = std, goalies = g, bios = bio
rate = 'n'# counts = n, rates = y, relative = r
team = 'ALL' #for individual teams use 3 character team code ie: New York Islanders = NYI
pos = 'S' #all skaters = S, forwards = F, center = C, right wing = R, left wing = L, defenseman = D, goalie = G
loc = 'B' #Home = 'H' #Away = 'A'
toi = 0 #minimum TOI filter
gpfilt = 'none' #This is param is for game range. by date = gpdate; by games played = gpteam
fd = '' #from date if gpfilt by date range (gpdate) 
td = '' #to date if gpfilt by date range (gpdate) 
tgp = 410   #last X team games played if gpfilt by games played (gpteam) 
lines = 'single' # pass 'split' to split player data by team if theyve played for multiple teams in range
draftteam = 'ALL' #pass 3 letter team acronym to specify player drafted by a given team

In [161]:
url = 'http://www.naturalstattrick.com/playerteams.php?fromseason={}&thruseason={}&stype={}&sit={}&score={}&stdoi={}&rate={}&team={}&pos={}&loc={}&toi={}&gpfilt={}&fd={}&td={}td&tgp={}&lines={}&draftteam={}'.format(
    fromseason,
    thruseason,
    stype, 
    sit,
    score, 
    stdoi, 
    rate, 
    team, 
    pos, 
    loc, 
    toi, 
    gpfilt, 
    fd, 
    td, 
    tgp, 
    lines, 
    draftteam)

In [162]:
print(url)

http://www.naturalstattrick.com/playerteams.php?fromseason=20202021&thruseason=20202021&stype=2&sit=5v5&score=all&stdoi=g&rate=n&team=ALL&pos=S&loc=B&toi=0&gpfilt=none&fd=&td=td&tgp=410&lines=single&draftteam=ALL


In [163]:
df3 = pd.read_html(url, header=0, index_col = 0, na_values=["-"])[0]
df3.head()

,Player,Team,GP,TOI,Shots Against,Saves,Goals Against,SV%,GAA,GSAA,...,LD Shots Against,LD Saves,LD Goals Against,LDSV%,LDGAA,LDGSAA,Rush Attempts Against,Rebound Attempts Against,Avg. Shot Distance,Avg. Goal Distance
1,Craig Anderson,WSH,2,66.250000,26,23,3,0.885,2.72,-0.88,...,8,8,0,1.000,0.00,0.19,2,3,32.31,16.00
2,Ryan Miller,ANA,5,243.316667,115,103,12,0.896,2.96,-2.61,...,40,38,2,0.950,0.49,-1.05,8,14,32.55,21.92
3,Mike Smith,EDM,10,440.250000,208,193,15,0.928,2.04,1.98,...,95,93,2,0.979,0.27,0.27,15,19,34.39,20.07
4,Curtis McElhinney,T.B,4,174.466667,65,57,8,0.877,2.75,-2.69,...,31,30,1,0.968,0.34,-0.26,6,3,37.49,18.00
5,Marc-Andre Fleury,VGK,14,698.366667,338,317,21,0.938,1.80,6.59,...,142,140,2,0.986,0.17,1.39,17,41,35.52,18.14


### Conclusion ###

You now have the tools to grab data from the tables on Natural Stat Trick and other similar websites. Give pd.read_html a shot first and if that doesn't work you can go step by step with the Beautiful Soup library. You can also dynamically update the URL parameters to get different types of data. 

Although the purveyor of Natural Stat Trick allows access to the data to the public for free, if you do find yourself using the data frequently, consider subscribing to the Patreon to support the continued operations of the site. 


Sources:
- http://www.naturalstattrick.com/
- https://requests.readthedocs.io/en/master/user/quickstart/
- https://theathletic.com/415611/2018/07/05/an-advanced-stats-primer-with-naturalstattricks-brad-timmins/
- https://www.crummy.com/software/BeautifulSoup/bs4/doc/
- https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html#reading-html-content